# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121829e+02     1.623583e+00
 * time: 0.09093284606933594
     1     1.154069e+01     8.901053e-01
 * time: 0.9873569011688232
     2    -1.186656e+01     1.066251e+00
 * time: 1.121025800704956
     3    -3.419200e+01     7.513000e-01
 * time: 1.2857248783111572
     4    -4.795572e+01     5.055250e-01
 * time: 1.4410769939422607
     5    -5.710360e+01     1.821407e-01
 * time: 1.589684009552002
     6    -5.990840e+01     1.123619e-01
 * time: 1.7055978775024414
     7    -6.093044e+01     5.337322e-02
 * time: 1.8155059814453125
     8    -6.131731e+01     6.444137e-02
 * time: 1.9351599216461182
     9    -6.162212e+01     4.013443e-02
 * time: 2.0483808517456055
    10    -6.182104e+01     3.124235e-02
 * time: 2.165247917175293
    11    -6.197434e+01     2.013354e-02
 * time: 2.281315803527832
    12    -6.203481e+01     1.619426e-02
 * time: 2.3905699253082275
    13    -6.209272e+01     1.452902e-02
 * time: 2.5064508914947

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557707
    AtomicNonlocal      14.8522661
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336824

    total               -62.261666461655
